In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np 
import requests
import gmaps
from api_keys import weather_api_key
from api_keys import g_key 

In [2]:
#csv file from WeatherPy project was saved and is being read
csv_df = pd.read_csv("HWOutput_data/cities.csv")
#resetting the index
csv_df.reset_index(drop= False , inplace=True)
#renaming index as a column(City_ID)
csv_renamed = csv_df.rename(columns={"index":"City_ID"})
csv_renamed.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,MaxTemp,WindSpeed
0,0,Luganville,90,VU,1570551458,83,-15.51,167.18,78.80,11.41
1,1,Isangel,20,VU,1570551459,94,-19.55,169.27,68.00,12.53
2,2,Beloha,0,MG,1570551459,21,-25.17,45.06,90.33,12.12
3,3,Bluff,50,AU,1570551460,27,-23.58,149.07,74.68,9.40
4,4,Xian,75,ES,1570551461,93,42.76,-8.49,69.01,9.17


In [3]:
#Creating a heatmap

gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = csv_renamed[["Lat", "Lng"]]
humidity = csv_renamed["Humidity"]

#plot map
fig = gmaps.figure()

#create heat layer
heatlayer = gmaps.heatmap_layer(locations, weights = humidity,dissipating=False, max_intensity = 100,point_radius=3) 

# Add layer
fig.add_layer(heatlayer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#cleaning the data by dropping rows with no values
csv_renamed.count()
cleancsv =csv_renamed.dropna(how="any")
cleancsv.count()

City_ID       536
City          536
Cloudiness    536
Country       536
Date          536
Humidity      536
Lat           536
Lng           536
MaxTemp       536
WindSpeed     536
dtype: int64

In [5]:
#Creating dataframe with ideal weather - temp <80F & >70F , windspeed <10mph and cloudiness =0

csv_80 = cleancsv.loc[cleancsv['MaxTemp'] >= 70,:]
csv_idealtemp = csv_80.loc[csv_80['MaxTemp']<=80,:]
csv_wind = csv_idealtemp.loc[csv_idealtemp['WindSpeed']< 10,:]
hotel_df = csv_wind.loc[csv_wind['Cloudiness'] == 0,:].copy()
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,MaxTemp,WindSpeed
11,11,Perpignan,0,FR,1570551465,73,42.70,2.88,75.99,9.17
149,149,Ruteng,0,ID,1570551505,88,-8.61,120.46,70.35,2.17
227,227,San Quintin,0,PH,1570551521,85,17.54,120.52,72.87,5.14
337,337,Birjand,0,IR,1570551547,17,32.86,59.22,73.40,6.93
355,355,Port Hedland,0,AU,1570551556,88,-20.31,118.58,73.40,5.82
405,405,Sainte-Maxime,0,FR,1570551566,43,43.31,6.64,72.00,8.05
428,428,Rome,0,IT,1570551571,56,41.89,12.48,75.20,6.93
451,451,Kadipur,0,IN,1570551576,54,26.16,82.38,79.54,2.74
463,463,Katherine,0,AU,1570551578,36,-14.47,132.26,78.80,6.45
468,468,Tanete,0,ID,1570551579,86,-3.94,119.78,71.80,2.30


In [6]:
#adding a new empty column to receive hotel names from API calls 

hotel_df["HotelName"] =""
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,MaxTemp,WindSpeed,HotelName
11,11,Perpignan,0,FR,1570551465,73,42.70,2.88,75.99,9.17,
149,149,Ruteng,0,ID,1570551505,88,-8.61,120.46,70.35,2.17,
227,227,San Quintin,0,PH,1570551521,85,17.54,120.52,72.87,5.14,
337,337,Birjand,0,IR,1570551547,17,32.86,59.22,73.40,6.93,
355,355,Port Hedland,0,AU,1570551556,88,-20.31,118.58,73.40,5.82,


In [7]:
#base url for  making google api calls for place - nearby search

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#parameters needed for the search - needs location parameter that will be added via the for loop
params = {"radius":5000, "type": "lodging", "key": g_key}

#iterating through each row in columns "Lat" and "Lng" to get latitude and longitude respectively
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # adding  "location" parameter to params
    params['location']= f"{lat},{lng}"
    
   # making the api call
    response = requests.get(base_url,params=params).json() 
   
    try:
        #getting name of the hotel from the json response which is a list of data and storing in a variable
        results = response["results"][0]["name"]
        # adding the hotel name to each row in the HotelName column
        hotel_df.loc[index,"HotelName"]= results
    except (KeyError, IndexError):
        hotel_df.loc[index,"HotelName"]="No Hotel"
        print("Missing field/result.....skipping")
       
hotel_df
             

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,MaxTemp,WindSpeed,HotelName
11,11,Perpignan,0,FR,1570551465,73,42.70,2.88,75.99,9.17,Comfort Hotel Centre Del Mon - Perpignan
149,149,Ruteng,0,ID,1570551505,88,-8.61,120.46,70.35,2.17,Hotel FX72 Ruteng
227,227,San Quintin,0,PH,1570551521,85,17.54,120.52,72.87,5.14,San Quintin River Resort
337,337,Birjand,0,IR,1570551547,17,32.86,59.22,73.40,6.93,تاکسی سرویس آرامش
355,355,Port Hedland,0,AU,1570551556,88,-20.31,118.58,73.40,5.82,The Esplanade Hotel
405,405,Sainte-Maxime,0,FR,1570551566,43,43.31,6.64,72.00,8.05,Hôtel Les Jardins De Sainte Maxime
428,428,Rome,0,IT,1570551571,56,41.89,12.48,75.20,6.93,Hotel Cosmopolita Roma
451,451,Kadipur,0,IN,1570551576,54,26.16,82.38,79.54,2.74,Manoj Kumar
463,463,Katherine,0,AU,1570551578,36,-14.47,132.26,78.80,6.45,Pine Tree Motel
468,468,Tanete,0,ID,1570551579,86,-3.94,119.78,71.80,2.30,Wisma Tri Multi


In [8]:
# since the csv file for the dataframe used in this project is created from using random lat and lng combinations from 
# previous project there is a chance that some cities may not have hotels .
# dropping rows with no hotel names
hotel_new = hotel_df[hotel_df.HotelName != "No Hotel"]
hotel_new

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,MaxTemp,WindSpeed,HotelName
11,11,Perpignan,0,FR,1570551465,73,42.70,2.88,75.99,9.17,Comfort Hotel Centre Del Mon - Perpignan
149,149,Ruteng,0,ID,1570551505,88,-8.61,120.46,70.35,2.17,Hotel FX72 Ruteng
227,227,San Quintin,0,PH,1570551521,85,17.54,120.52,72.87,5.14,San Quintin River Resort
337,337,Birjand,0,IR,1570551547,17,32.86,59.22,73.40,6.93,تاکسی سرویس آرامش
355,355,Port Hedland,0,AU,1570551556,88,-20.31,118.58,73.40,5.82,The Esplanade Hotel
405,405,Sainte-Maxime,0,FR,1570551566,43,43.31,6.64,72.00,8.05,Hôtel Les Jardins De Sainte Maxime
428,428,Rome,0,IT,1570551571,56,41.89,12.48,75.20,6.93,Hotel Cosmopolita Roma
451,451,Kadipur,0,IN,1570551576,54,26.16,82.38,79.54,2.74,Manoj Kumar
463,463,Katherine,0,AU,1570551578,36,-14.47,132.26,78.80,6.45,Pine Tree Motel
468,468,Tanete,0,ID,1570551579,86,-3.94,119.78,71.80,2.30,Wisma Tri Multi


In [9]:
#  starting with the previous heat map and for each hotel adding markers and maker layer with info box 

gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = csv_renamed[["Lat", "Lng"]]
humidity = csv_renamed["Humidity"]

#plot map
fig = gmaps.figure()

#create heat layer
heatlayer = gmaps.heatmap_layer(locations, weights = humidity,dissipating=False, max_intensity = 100,point_radius=3) 

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{HotelName}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_new.iterrows()]
locations_hotel = hotel_new[["Lat", "Lng"]]
#create the marker layer
marker_layer = gmaps.marker_layer(locations_hotel, info_box_content=hotel_info)

# Add heat layer and marker layer
fig.add_layer(heatlayer)
fig.add_layer( marker_layer )
fig

Figure(layout=FigureLayout(height='420px'))